In [1]:
# for speech-to-text
import speech_recognition as sr
# for text-to-speech
from gtts import gTTS
# for language model
import transformers
import os
import time
# for data
import os
import datetime
import numpy as np
import tempfile

In [2]:
class ChatBot():
  def __init__(self,name):
    print("------get ready to meet",name,"------")
    self.name = name
  def speech_to_text(self):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Adjusting noise ")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        print("Listening....")
        recorded_audio = recognizer.listen(source, timeout=4)
        # print("Done recording")
        self.text = "ERROR"

    try:
        # print("Recognizing the text")
        self.text = recognizer.recognize_google(
                recorded_audio, 
                language="en-US"
            )
        print("me -->: {}".format(self.text))
    except Exception as ex:
        print(ex)
  @staticmethod
  def text_to_speech(self,text):
        print("jarvis --> ", text)
        speaker = gTTS(text=text, lang="en", slow=False)
        speaker.save("wel.flac")
        statbuf = os.stat("wel.flac")
        mbytes = statbuf.st_size / 1024
        duration = mbytes / 200
        # playsound.playsound('E:/sem6/NLP/chatbot_transfer_learning/audio.mp3')
        os.system('start wel.flac')  #if you are using mac->afplay or else for windows->start
        os.system("close wel.flac")
        time.sleep(int(50*duration))
        os.remove("wel.flac")
  def wake_up(self, text):
    return True if self.name in text.lower() else False
  @staticmethod
  def action_time():
    return datetime.datetime.now().time().strftime('%H:%M')

In [ ]:
if __name__ == "__main__":
    ai = ChatBot(name='jarvis')
    nlp = transformers.pipeline(
        "conversational",
        model="microsoft/DialoGPT-medium",
        tokenizer="microsoft/DialoGPT-medium",
    )
    os.environ["TOKENIZERS_PARALLELISM"] = "true"
    ex = True
    while ex:
        ai.speech_to_text()  # capture user input
        # wake up to call
        if ai.wake_up(ai.text) is True:
            res = "Hello i am jarvis the AI , what can i do for you?"
            ai.text_to_speech(res)
        # any action as time
        elif "time" in ai.text:
            res = ai.action_time()
            ai.text_to_speech(res)
        # polite response to thanks
        elif any(i in ai.text for i in ['thanks', 'thank', 'thankyou']):
            res = np.random.choice(["you're welcome!", "anytime", "no worries", "cool", "bas kya yaar!"])
            ai.text_to_speech(res)
        elif any(i in ai.text for i in ["exit", "close", "bye"]):
            res = np.random.choice(["Tata", "Have a good day", "Bye", "Goodbye", "see you soon!"])
            ai.text_to_speech(res)
            ex = False
        # convo
        else:
            if ai.text == "ERROR":
                res = "Sorry, come again!"
            else:
                chat = nlp(transformers.Conversation(ai.text), pad_token_id=50256)
                res = str(chat)
                res = res[res.find("bot >>") + 6:].strip()
                ai.text_to_speech(res)
    print("-----Jarvis shutting down-----")
